In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from datetime import datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from geourban import services, types
import pandas as pd

In [ ]:
class UrbanSimulation(object):
    
    def __init__(self):
        self._client = EnvironmentClientFactory.create_client_with_host('geourban.p.rapidapi.com')
        self._simulations = services.simulations(self._client)
    
    def find_simulation(self, region_name):
        for simulation in self._simulations:
            if region_name == simulation['name']:
                return simulation
        
        raise ValueError(f'No traffic simulation for "{region_name}" exists!')
    
    def query_car_hotspots(self, region_code, simulation_date):
        vehicle_type = types.VehicleType.CAR
        grid_type = types.GridType.AGENT
        traffic_grid = services.top(self._client, region_code, simulation_date, vehicle_type, grid_type)
        traffic_grid_sdf = FeatureSet.from_geojson(traffic_grid).sdf
        traffic_grid_sdf[['start_time', 'end_time']] = traffic_grid_sdf[['start_time', 'end_time']].apply(pd.to_datetime)
        traffic_grid_sdf['agent_count'] = traffic_grid_sdf['agent_count'].apply(pd.to_numeric)
        return traffic_grid_sdf[['start_time', 'end_time', 'agent_count', 'SHAPE']]
    
    def query_car_positions(self, latitude, longitude, simulation_datetime):
        vehicle_type = types.VehicleType.CAR
        agent_positions = services.query(self._client, simulation_datetime, vehicle_type, latitude, longitude)
        agent_positions_sdf = FeatureSet.from_geojson(agent_positions).sdf
        agent_positions_sdf[['trip', 'person']] = agent_positions_sdf[['trip', 'person']].apply(pd.to_numeric)
        agent_positions_sdf['trip_time'] = agent_positions_sdf['trip_time'].apply(pd.to_datetime)
        return agent_positions_sdf[['trip', 'person', 'trip_time', 'SHAPE']]

In [ ]:
urban_sim = UrbanSimulation()
bonn_simulation = urban_sim.find_simulation('Bonn, Kreisfreie Stadt')
region_code = bonn_simulation['region']
simulation_date = datetime.fromisoformat(bonn_simulation['date']).date()
car_speed_spatial_df = urban_sim.query_car_hotspots(region_code, simulation_date)
gis = GIS()
bonn_map = gis.map('Bonn, Germany', zoomlevel=13)
car_speed_spatial_df.spatial.plot(bonn_map, renderer_type='s', colors='#E80000', alpha=0.3)
bonn_map

In [ ]:
urban_sim = UrbanSimulation()
(latitude, longitude) = (50.746708, 7.074405)
simulation_datetime = datetime.fromisoformat('2023-08-24T08:00:00')
car_positions_spatial_df = urban_sim.query_car_positions(latitude, longitude, simulation_datetime)
car_positions_spatial_df

In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

from arcgis.features import FeatureLayer
import pytz

class UrbanSimulationPortal(object):
    
    def __init__(self):
        self._gis = None
    
    def login(self, username):
        """
        Login into ArcGIS Online using a password input.

        :param str username: Your ArcGIS Online username.
        """
        self._gis = GIS(username=username)
        
    def publish_positions(self, positions_sdf, service_name, folder):
        """
        Publish agent positions as a feature service.

        :param SDF positions_sdf: The geospatial features to publish.
        :param str service_name: The name of the feature service.
        :param str folder: The name of the existing output folder.
        """
        agent_positions_sdf = positions_sdf.copy()
        
        # Publish only the first position as feature layer
        portal_item = agent_positions_sdf[:1].spatial.to_featurelayer(service_name, folder=folder)
        
        # Access the underlying feature layer from the newly created portal item
        if len(portal_item.layers) < 1:
            raise ValueError(f"Feature service '{service_name}' is not valid!")
            
        feature_layer = portal_item.layers[0]
        feature_layer.delete_features(where='1=1')
        agent_positions_fset = agent_positions_sdf.spatial.to_featureset()
        agent_positions_features = agent_positions_fset.features
        feature_layer.edit_features(adds=agent_positions_features)
        return feature_layer

In [ ]:
def convert_triptimes_to_utc(positions_sdf, wk_timezone):
    agent_positions_sdf = positions_sdf.copy()
    
    if not None is wk_timezone:
        if not 'trip_time' in agent_positions_sdf.columns:
            raise ValueError("Column 'trip_time' is missing!")
                
        agent_positions_sdf['trip_time'] = agent_positions_sdf['trip_time'].apply(lambda trip_time: trip_time.tz_localize(wk_timezone))
        agent_positions_sdf['trip_time'] = agent_positions_sdf['trip_time'].apply(lambda trip_time: trip_time.tz_convert(None))
        
    return agent_positions_sdf

In [ ]:
wk_timezone = 'Europe/Berlin'
convert_triptimes_to_utc(car_positions_spatial_df, wk_timezone)

In [ ]:
import os

In [ ]:
urban_portal = UrbanSimulationPortal()
username = os.environ.get('arcgis_user')
urban_portal.login(username)
car_feature_layer = urban_portal.publish_positions(car_positions_spatial_df, 'bonn_cars', 'Urban')
car_feature_layer